In [1]:
import os
import numpy as np
import pandas as pd
from glob import glob

In [2]:
img_paths = glob("../data/Images/*")

In [3]:
data = pd.read_csv("../data/data.csv")
data.shape

(4206, 5)

In [4]:
data.head()

,Unnamed: 0,bmi,gender,is_training,name
0,0,34.207396,Male,1,img_0.bmp
1,1,26.453720,Male,1,img_1.bmp
2,2,34.967561,Female,1,img_2.bmp
3,3,22.044766,Female,1,img_3.bmp
4,4,37.758789,Female,1,img_4.bmp


In [5]:
del data["Unnamed: 0"]

In [6]:
data.head()

,bmi,gender,is_training,name
0,34.207396,Male,1,img_0.bmp
1,26.453720,Male,1,img_1.bmp
2,34.967561,Female,1,img_2.bmp
3,22.044766,Female,1,img_3.bmp
4,37.758789,Female,1,img_4.bmp


In [14]:
data.gender.value_counts()

Male      2438
Female    1768
Name: gender, dtype: int64

In [15]:
import keras_vggface

ModuleNotFoundError: No module named 'tensorflow'

In [7]:
import torchvision.models as models
import pickle
import torch
from torchvision import transforms
from PIL import Image
from torch.autograd import Variable
import numpy as np
from tqdm.notebook import tqdm

In [8]:
model = models.resnet50(num_classes=8631,pretrained=False)
with open('../models/resnet50_ft_weight.pkl', 'rb') as f:
    obj = f.read()

weights = {key: torch.from_numpy(arr) for key, arr in pickle.loads(obj).items()}
model.load_state_dict(weights)

scaler = transforms.Resize((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
to_tensor = transforms.ToTensor()

model.eval()

/home/halo/miniconda3/envs/face2bmi/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/halo/miniconda3/envs/face2bmi/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [24]:
len(img_paths)-8

3955

In [31]:
# features = []
for imagepath in tqdm(img_paths):
    im_name = imagepath.split("/")[-1].split(".")[0]
    image = Image.open(imagepath)
    try:
        imgblob = Variable(normalize(to_tensor(scaler(image))).unsqueeze(0))
        tf_last_layer_chopped = torch.nn.Sequential(*list(model.children())[:-1])
        output = tf_last_layer_chopped(imgblob)
        with open(f"../output/{im_name}.npy", "w") as f:
            f.write(output[0, :, 0, 0].tolist())
            
    except Exception as e:
        print(imagepath, e)
        print()

# features = np.array(features)

  0%|          | 0/3963 [00:00<?, ?it/s]

../data/Images/img_1390.bmp write() argument must be str, not list

../data/Images/img_526.bmp write() argument must be str, not list

../data/Images/img_582.bmp write() argument must be str, not list

../data/Images/img_3599.bmp write() argument must be str, not list

../data/Images/img_1966.bmp write() argument must be str, not list

../data/Images/img_2698.bmp write() argument must be str, not list

../data/Images/img_289.bmp write() argument must be str, not list

../data/Images/img_1135.bmp write() argument must be str, not list

../data/Images/img_2252.bmp write() argument must be str, not list

../data/Images/img_527.bmp write() argument must be str, not list

../data/Images/img_325.bmp write() argument must be str, not list

../data/Images/img_3239.bmp write() argument must be str, not list

../data/Images/img_840.bmp write() argument must be str, not list

../data/Images/img_2023.bmp write() argument must be str, not list

../data/Images/img_2441.bmp write() argument must be s

KeyboardInterrupt: 

In [30]:
Image.open("../data/Images/img_169.bmp").array()

AttributeError: array